In [1]:
from datasets import load_dataset
import Preprocessor as p
import numpy as np
import nltk
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from transformers import pipeline
from datasets import load_dataset
import torch
from tqdm import tqdm
import re
gen = pipeline('text-generation', model='gpt2')
import csv
from transformers import AutoModelForCausalLM, AutoTokenizer
from helpMe import get_ppl, convert_to_float, getList, Average
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
from customDataset import CustomDataset

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/oliver/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [2]:
# Load the copenlu/answerable_tydiqa dataset
dataset = load_dataset("copenlu/answerable_tydiqa")

train_df = dataset['train'].to_pandas()
train_df = train_df[train_df['language'].isin(['indonesian', 'arabic', 'bengali'])]
train_df = p.DataFramePreprocessor(train_df).df

val_df = dataset['validation'].to_pandas()
val_df = val_df[val_df['language'].isin(['indonesian', 'arabic', 'bengali'])]
val_df = p.DataFramePreprocessor(val_df).df


In [3]:
data = train_df
data['document_length'] = data['document_plaintext'].str.len()
data['is_answerable'] = data['annotations'].apply(lambda x: x.get('answer_start', [-1])[0] != -1)

In [4]:
fst_prompt = train_df.iloc[0]['question_text']
from translate import Translator
translator_in = Translator(to_lang="en", from_lang = "bn")
translator_out = Translator(to_lang="bn", from_lang = "en")
translation = translator_in.translate(fst_prompt).split()
text = " ".join(translation[:4])
out = gen(text,num_return_sequences = 3, max_length = len(translation)+1)
print(out[0],"\n")
print(out[1],"\n")
print(out[2],"\n")
print("original text: " + " ".join(translation))

/home/oliver/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'generated_text': 'Genghis Khan was the founder of Khan'} 

{'generated_text': 'Genghis Khan was the first to admit'} 

{'generated_text': 'Genghis Khan was the youngest son of'} 

original text: Genghis Khan was the king of which clan?


In [5]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [6]:
#inputs = tokenizer(data['question_text'][3], return_tensors = "pt")
#loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
#ppl = torch.exp(loss)
#print(ppl)



"""inputs_wiki_text = tokenizer("Generative Pretrained Transformer is an opensource artificial intelligence created by OpenAI in February 2019", return_tensors = "pt")
loss = model(input_ids = inputs_wiki_text["input_ids"], labels = inputs_wiki_text["input_ids"]).loss
ppl = torch.exp(loss)
print(ppl)"""


'inputs_wiki_text = tokenizer("Generative Pretrained Transformer is an opensource artificial intelligence created by OpenAI in February 2019", return_tensors = "pt")\nloss = model(input_ids = inputs_wiki_text["input_ids"], labels = inputs_wiki_text["input_ids"]).loss\nppl = torch.exp(loss)\nprint(ppl)'

In [7]:
bengali = data[data['language'] == 'bengali']
arabic = data[data['language'] == 'arabic']
indonesian = data[data['language'] == 'indonesian']

In [8]:
#ben = get_ppl(bengali, "bengali")
#ara = get_ppl(arabic, "arabic")
#ind = get_ppl(indonesian, "indonesian")

In [9]:
ppl_a = pd.read_csv('arabic.csv')  
ppl_b = pd.read_csv('bengali.csv')  
ppl_i = pd.read_csv('indonesian.csv')  

In [10]:
ppl_ara = getList('arabic.csv')[0]
ppl_ben = getList('bengali.csv')[0]
ppl_ind = getList('indonesian.csv')[0]


In [11]:
ar = Average(convert_to_float(ppl_ara))
be = Average(convert_to_float(ppl_ben))
ind = Average(convert_to_float(ppl_ind))

In [12]:
print(ar)
print(be)
print(ind)

20.281176299731527
5.924668006843454
3924.599611058437


In [30]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.Dataset.iloc[index]['document_plaintext']
        label = self.Dataset.iloc[index]['is_answerable']
        
        # Convert the label to 0 if False, 1 if True
        label = 1 if label else 0
        
        return text, label


In [32]:


# Define hyperparameters
batch_size = 2  # You can adjust this based on your needs

# Create an instance of your custom dataset
bengali = bengali[['document_plaintext','is_answerable']].iloc[1:]
custom_dataset = CustomDataset(bengali)
test_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [28]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# Encode text and generate embeddings
text = bengali.iloc[1]['document_plaintext']

input_ids = tokenizer.encode(text, return_tensors='pt')
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # You can use mean pooling for sentence embeddings
